# DL vertex input image generation

This notebook is designed to take CSV files generated by the <code>PrepareTrainingSample</code> function of the <code>DlVertexingAlgorithm</code>. This algorithm generates CSV files for each of the U. V and W views and the code below will run over each of those files.
    
Most of the cells below will not need any editing, but at the very bottom of the notebook you will find some additional markdown that describes what you may need to edit (essentially just some file locations).

In [ ]:
# Automatically reload external libraries that change
%reload_ext autoreload
%autoreload 2

# If a matplotlib plot command is issued, display the results in the notebook
%matplotlib inline

In [ ]:
def get_range(x_span, z_span, image_size):
    image_height, image_width = image_size
    x_min, x_max = x_span
    z_min, z_max = z_span
    x_range = x_max - x_min
    z_range = z_max - z_min
    
    if 2 * x_range < image_width:
        padding = 0.5 * (image_width / 2. - x_range)
        x_min -= padding
        x_max += padding
        x_range = x_max - x_min
    if 2 * z_range < image_height:
        padding = 0.5 * (image_height / 2. - z_range)
        z_min -= padding
        z_max += padding
        z_range = z_max - z_min
    
    return (x_min, x_max, z_min, z_max)

In [ ]:
def make_input_histogram(x, z, adc, vertex, image_size):
    global thresholds
    image_height, image_width = image_size
    x_min, x_max = np.amin(x), np.amax(x)
    z_min, z_max = np.amin(z), np.amax(z)
    
    # Update the span if image is too small
    x_min, x_max, z_min, z_max = get_range((x_min, x_max), (z_min, z_max), image_size)
    r_span = np.sqrt((x_max - x_min)**2 + (z_max - z_min)**2)
    
    eps = np.finfo(np.float32).eps
    x_bins = np.linspace(x_min - eps, x_max + eps, image_width + 1)
    z_bins = np.linspace(z_min - eps, z_max + eps, image_height + 1)

    phx = np.digitize(x, x_bins) - 1
    phz = np.digitize(z, z_bins) - 1
    pha = np.digitize(adc, z_bins) - 1
    
    pvx = np.digitize(vertex[0], x_bins) - 1
    pvz = np.digitize(vertex[1], z_bins) - 1

    input_histogram, _, _ = np.histogram2d(z, x, bins=[z_bins, x_bins], weights=adc)
    input_histogram = 255. * input_histogram / np.max(input_histogram)
    input_histogram = input_histogram.astype(np.uint8)
    # Flip to ensure z = 0 is at the bottom (cv2.imwrite assumes [row, col] ordering)
    input_histogram = np.flip(input_histogram, axis=0)

    # ATTN: Need to set the dtype here or truth will be of type 8-bit uint before scaling, which will result in
    # incorrect distances for dr > 255
    truth_histogram = np.zeros_like(input_histogram, dtype=np.float)
    
    # Handle case where vertex is outside of the hit bounding box. Note, it may still be possible for the correction
    # to move the vertex outside the bounding box in the opposite direction (unlikely, but think about it)
    # other way to do this is to bound the image including the vertex location
    if pvx < 0: # underflow
        pvx = -(np.digitize(x_bins[0] + (x_bins[0] - vertex[0]), x_bins) - 1)
    elif pvx >= (len(x_bins) - 1): # overflow
        pvx = np.digitize(x_bins[-1] - (vertex[0] - x_bins[-1]), x_bins) - 1
    if pvz < 0: # underflow
        pvz = -(np.digitize(z_bins[0] + (z_bins[0] - vertex[1]), z_bins) - 1)
    elif pvz >= (len(z_bins) - 1): # overflow
        pvz = np.digitize(z_bins[-1] - (vertex[1] - z_bins[-1]), z_bins) - 1

    dr = np.sqrt((phx - pvx)**2 + (phz - pvz)**2)
    class_histogram = np.zeros_like(truth_histogram)
    for i in range(len(phx)):
        truth_histogram[phz[i], phx[i]] = dr[i]
    truth_min, truth_max = np.min(truth_histogram), np.max(truth_histogram)
    if truth_max > truth_min:
        truth_histogram = (truth_histogram - truth_min) / np.ceil(np.sqrt(2*(image_height - 1)**2))
        for i in range(len(phx)):
            cls = np.digitize(truth_histogram[phz[i], phx[i]], thresholds)
            class_histogram[phz[i], phx[i]] = cls if cls < len(thresholds) else len(thresholds) - 1
    else:
        class_histogram = np.zeros_like(input_histogram)
    class_histogram = class_histogram.astype(np.uint8)
    # Flip to ensure z = 0 is at the bottom (cv2.imwrite assumes [row, col] ordering)
    class_histogram = np.flip(class_histogram, axis=0)

    return input_histogram, class_histogram

In [ ]:
import cv2
import csv
import numpy as np
import os


def display(hits_x, hits_z, vrt_x, vrt_z):
    """Displays an image of the hits and vertices.
    
        Args:
            hits_x: list of x coordinates for hits
            hits_z: list of z coordinates for hits
            vrt_x: list of x coordinates for vertices
            vrt_z: list of z coordinates for vertices
    """
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(5,5))
    plt.scatter(hits_x, hits_z, c='black', s=20, alpha=1.0, label="Hits")
    plt.scatter(vrt_x, vrt_z, c='red', s=50, alpha=1.0, label="Vertices")
    plt.legend()
    plt.show()
    plt.close(fig)

def process_file(input_file, output_folder, image_size = (128, 128)):
    """Generate the training/validation set images for events.
    
        The input CSV file has the format:
        Date/Time,N Vertices,M Hits,N*{vertex x, vertex y},M*{hit x, hit y},EOL
        
        The first vertex in the list is always the primary vertex
    
        Args:
            input_file: a CSV file containing event information
            output_folder: the top-level folder for output images
            image_size: the output image size as a tuple (height, width)
    """
    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            data = row[1:-1]
            process_event(data, output_folder, f"{i}", image_size)

def process_event(data, output_folder, event, image_size):
    """Generate the training/validation set images for a single event.
    
        The input data has the format:
        N Vertices,M Hits,N*{vertex x, vertex y},M*{hit x, hit z, adc}
        
        The first vertex in the list is always the primary vertex
        
        Images are output to <output_folder>/Hits and <output_folder>/Truth
    
        Args:
            data: the set of vertices and hits for the event
            output_folder: the top-level folder for output images
            event: the event number
            image_size: the output image size as a tuple (height, width)
    """
    nv_coords = 2
    nh_coords = 3
    nuance = int(data.pop(0))
    n_vertices = int(data.pop(0))
    n_hits = int(data.pop(0))
    length = len(data)
    
    if length != (n_vertices * nv_coords + n_hits * nh_coords):
        print('Missing information in input file')
        print(n_vertices, n_hits, length)
        return
    
    v_start, v_finish = 0, nv_coords * n_vertices
    vx, vz = np.array(data[v_start:v_finish:2], dtype=np.float), np.array(data[v_start + 1:v_finish:2], dtype=np.float)
    
    h_start, h_finish = nv_coords * n_vertices, nv_coords * n_vertices + nh_coords * n_hits
    hx = np.array(data[h_start:h_finish:nh_coords], dtype=np.float)
    hz = np.array(data[h_start + 1:h_finish:nh_coords], dtype=np.float)
    hadc = np.array(data[h_start + 2:h_finish:nh_coords], dtype=np.float)

    if hx.size == 0 or hz.size == 0 or hadc.size == 0:
        return
    
    input_histogram, truth_histogram = make_input_histogram(hx, hz, hadc, (vx[0], vz[0]), image_size)
    
    hits_output_folder = os.path.join(output_folder, "Hits")
    hits_filename = os.path.join(hits_output_folder, f"Image_{event}.png")
    cv2.imwrite(hits_filename, input_histogram)
    
    truth_output_folder = os.path.join(output_folder, "Truth")
    truth_filename = os.path.join(truth_output_folder, f"Image_{event}.png")
    cv2.imwrite(truth_filename, truth_histogram)

# Edit below this point

The details that might change between different contexts are the input and output file locations, the class thresholds and potentially the number of passes.

The input files are specified by the <code>file_prefix</code> variable - the <code>PrepareTrainingSample</code> function automatically tags the files with their respective views, so you should omit the view and file type from the specification.

The output location is specified by <code>global_path</code>, within which <code>Hit</code> and <code>Truth</code> folders will be created to store the input and target output images for training.

The thresholds are specified by the <code>thresholds</code> variable, a global variable referenced by <code>make_input_histograms</code>. It is critical that the values here match those specified in the Pandora XML specification for the vertexing algorithm.

In general, you will want to implement a two pass approach to networking, as this will likely greatly enhance vertex resolution, but if you only want one pass (or perhaps only need to retrain the networks for one of the passes), you can just alter the <code>vertex_pass</code> loop list to run over the selected pass, as there is no dependency between the passes for image generation and training purposes.

Once you're happy with these values, you can just run the entire notebook from top to bottom and, after some time, you'll have a set of input/truth images that can be used to train the networks.

In [ ]:
thresholds = [0., 0.00275, 0.00825, 0.01925, 0.03575, 0.05775, 0.08525, \
              0.12375, 0.15125, 0.20625, 0.26125, 0.31625, 0.37125, 0.42625, \
              0.50875, 0.59125, 0.67375, 0.75625, 0.85, 1.0]

for vertex_pass in [1 , 2]:
    #file_prefix = f"csv/DUNEFD_MC11_Pass{vertex_pass}_CaloHitList"
    file_prefix = f"atmospherics_csv/DUNEFD_MC11_Atmos_Pass{vertex_pass}_CaloHitList"
    image_size = (256, 256) if vertex_pass == 1 else (128, 128)
    from tqdm.notebook import tqdm
    for view in tqdm(["W", "U", "V"], desc="Processing views"):
        #global_path = os.path.join(f"Pass{vertex_pass}", f"Images{view}")
        global_path = os.path.join("Atmos", f"Pass{vertex_pass}", f"Images{view}")
        for path in [os.path.join(global_path, "Hits"), os.path.join(global_path, "Truth")]:
            if not os.path.exists(path):
                os.makedirs(path)
        process_file(file_prefix + view + ".csv", global_path, image_size = image_size)